# Xarray/Zarr/Icechunk on S3

You will need to run this notebook in a `conda` environment created from `environment.yml`.

In [ ]:
import zarr
from icechunk import Repository, s3_storage

## Create a new Zarr store backed by Icechunk

This example uses a S3 store

In [ ]:
s3_storage = s3_storage(
    bucket="icechunk-test", prefix="oscar-demo-repository", from_env=True
)

In [ ]:
repo = Repository.create(
    storage=s3_storage,
)

## Real data

In [ ]:
import xarray as xr

In [ ]:
import fsspec

fs = fsspec.filesystem("s3")

In [ ]:
oscar = xr.open_dataset(
    fs.open("s3://earthmover-sample-data/netcdf/oscar_vel2018.nc"),
    chunks={},
    engine="h5netcdf",
)
oscar

## Write to icechunk

In [ ]:
session = repo.session("main")

group = zarr.group(store=session.store, overwrite=True)
group

In [ ]:
import time

for var in oscar:
    session = repo.writable_session("main")
    group = zarr.open_group(store=session.store)
    print(var)
    tic = time.time()
    group.create_array(
        name=var,
        shape=oscar[var].shape,
        chunk_shape=(1, 1, 481, 1201),
        fill_value=-1234567,
        dtype=oscar[var].dtype,
        data=oscar[var],
        overwrite=True,
    )
    print(session.commit(f"wrote {var}"))
    print(f"committed; {time.time() - tic} seconds")

In [ ]:
main_snapshot_id = repo.lookup_branch("main")
repo.ancestry(main_snapshot_id)

## Open store

In [ ]:
import zarr
from icechunk import Repository, s3_storage

s3_storage = s3_storage(
    bucket="icechunk-test", prefix="oscar-demo-repository", from_env=True
)

In [ ]:
repo = Repository.open(
    storage=s3_storage,
)
repo

Look at history

In [ ]:
[s.message for s in repo.ancestry(branch="main")]

In [ ]:
session = repo.readonly_session(branch="main")
root_group = zarr.open_group(store=session.store)

In [ ]:
root_group.members()

In [ ]:
root_group.members()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(root_group["u"][20, 0, :, :], cmap=mpl.cm.RdBu_r, vmin=-0.5, vmax=0.5)
plt.gcf().set_size_inches((9, 5))
plt.colorbar(location="bottom", orientation="horizontal", shrink=0.5, aspect=30)